In [1]:
import pandas as pd 
import numpy as np
import math
import tensorflow as tf
print(pd.__version__)
import matplotlib.pyplot as plt
import progressbar
import scipy

1.2.0


## Print Dependencies



Dependences are fundamental to record the computational environment.

In [2]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,keras,numpy,math,tensorflow,matplotlib,h5py,progressbar,scipy

# date
print (" ")
%watermark -u -n -t -z

Python implementation: CPython
Python version       : 3.8.7
IPython version      : 7.18.1

pandas     : 1.2.0
keras      : 2.4.3
numpy      : 1.19.5
math       : unknown
tensorflow : 2.4.0
matplotlib : 3.3.3
h5py       : 2.10.0
progressbar: 2.5
scipy      : 1.5.2

Compiler    : Clang 12.0.0 (clang-1200.0.32.28)
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

 
Last updated: Tue Feb 02 2021 11:14:30CET



## Load of the data

In [48]:
from process import loaddata
class_data = loaddata("../data/classifier/250.csv")
regr_data = loaddata("../data/regression/250.csv")

In [52]:
no_int = []
for row in class_data:
    if row[0] != 1:
        no_int.append([row[1], row[2], row[3], row[4], row[5], row[6], row[4], row[5], row[6]])
no_int = np.array(no_int)

In [57]:
interacted = []
for row in regr_data:
    interacted.append([row[0], row[1], row[2], row[3], row[4], row[5], row[-3], row[-2], row[-1]])
interacted = np.array(interacted)

In [59]:
data = np.concatenate((no_int, interacted), axis=0)

In [66]:
np.random.shuffle(data)
y = data[:,-3:]
x = data[:,0:6]

In [78]:
x.shape

(334438, 6)

In [68]:
train_split = 0.75
train_limit = int(len(y)*train_split)
print("Training sample: {0} \nValuation sample: {1}".format(train_limit, len(y)-train_limit))

Training sample: 250828 
Valuation sample: 83610


In [69]:
x_train = x[:train_limit]
x_val = x[train_limit:]

y_train = y[:train_limit]
y_val = y[train_limit:]

## Model Build

In [70]:
from keras.models import Sequential
from keras.layers.core import Dense
import keras.backend as K
from keras import optimizers
from keras import models
from keras import layers
from keras import regularizers

## !! 
The dropout 0.2 seems to work better

In [74]:
def build_model() :
    model = models.Sequential()
    model.add (layers.Dense (6, kernel_initializer= "normal" , input_shape = x.shape))
    model.add (layers.Dense (12, activation = "relu"))
    model.add (layers.Dense (32, activation = "relu"))
    model.add (layers.Dense (64, activation = "relu"))
    model.add (layers.Dense (64, activation = "relu"))
    model.add (layers.Dense (128, activation = "relu"))
    model.add (layers.Dense (128, activation = "relu"))
    model.add (layers.Dense (64, activation = "relu"))
    model.add (layers.Dense (32, activation = "relu"))
    model.add (layers.Dense (12, activation = "relu"))
    model.add (layers.Dense (6, activation = "relu"))
    model.add (layers.Dense (3))
    model.compile(optimizer = "rmsprop" , loss = "mse" , metrics =["mape"])
    return model

In [77]:
model = build_model ()
history = model.fit ( x_train, y_train, epochs = 20, batch_size = 64, validation_data = (x_val, y_val) )
model.save("../models/classificationandregression/large_mse250.h5")

Epoch 1/20
3920/3920 [==============================] - 18s 4ms/step - loss: 0.0073 - mape: 121.4765 - val_loss: 0.0072 - val_mape: 134.2847
Epoch 2/20
3920/3920 [==============================] - 15s 4ms/step - loss: 0.0072 - mape: 120.0057 - val_loss: 0.0072 - val_mape: 125.9939
Epoch 3/20
3920/3920 [==============================] - 15s 4ms/step - loss: 0.0072 - mape: 121.8266 - val_loss: 0.0072 - val_mape: 119.7950
Epoch 4/20
3920/3920 [==============================] - 18s 5ms/step - loss: 0.0073 - mape: 123.4608 - val_loss: 0.0072 - val_mape: 114.1179
Epoch 5/20
3920/3920 [==============================] - 17s 4ms/step - loss: 0.0072 - mape: 126.0862 - val_loss: 0.0072 - val_mape: 108.9816
Epoch 6/20
2723/3920 [===================>..........] - ETA: 4s - loss: 0.0072 - mape: 133.0306

KeyboardInterrupt: 

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Test spectrum

In [ ]:
from tensorflow import keras
model = keras.models.load_model('../models/classificationandregression/large_mse250.h5')

In [ ]:
model.summary()

In [ ]:
def energy_spectrum(energy_array, bins):
    energy_array = np.array(energy_array)
    plt.hist(energy_array, bins, histtype=u'step', density=True)
    plt.yscale("log")
    plt.show()

In [ ]:
y

In [ ]:
from tensorflow import keras 
photon_final_nn = []
prediction = model.predict(x)

In [ ]:
p1e_nn = prediction[:,0] 
p1e = y[:,0]
print(p1e_nn)
print(p1e)
plt.hist(p1e_nn, 60, alpha=0.5, label='NN prediction', density = True)
plt.hist(p1e, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
plt.xlabel('Electron momentum x-direction')
plt.ylabel('count')
plt.legend(loc='upper right')
#plt.xlim((0, 0.25))
plt.ylim((0, 12))
plt.show()

In [ ]:
p2e_nn = prediction[:,1] 
p2e = y[:,1]
print(p2e_nn)
print(p2e)
plt.hist(p2e_nn, 60, alpha=0.5, label='NN prediction', density = True)
plt.hist(p2e, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
plt.xlabel('Electron momentum y-direction')
plt.ylabel('count')
plt.legend(loc='upper right')
#plt.xlim((0, 0.25))
plt.ylim((0, 12))
plt.show()

In [ ]:
p3e_nn = prediction[:,2] 
p3e = y[:,2]
print(p3e_nn)
print(p3e)
plt.hist(p3e_nn, 100, alpha=0.5, label='NN prediction', density = True)
plt.hist(p3e, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
plt.xlabel('Electron momentum z-direction')
plt.ylabel('count')
plt.legend(loc='upper right')
plt.ylim((0, 12))
plt.show()

In [ ]:
fig, axs = plt.subplots(3, sharex = True)
fig.suptitle('3-momentum electrons post interaction')
axs[0].hist(p1e_nn, 100, alpha=0.5, label='NN prediction', density = True)
axs[0].hist(p1e, 100, alpha=0.5, label='NN prediction', density = True)
axs[1].hist(p2e_nn, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
axs[1].hist(p2e, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
axs[2].hist(p3e_nn, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
axs[2].hist(p3e, 100, alpha=0.5, label='Photon Momentum from simulations', density = True)
fig.show()

In [ ]:
final_e = []
final_e_nn = []
for classi_ in class_data:
    final_e.append(np.linalg.norm(classi_[-7:-4]))

In [ ]:
from tensorflow import keras 
final_e_nn = []
threshold = 0.5
num = int(1*len(x))
bar = progressbar.ProgressBar(maxval=num, 
                              widgets=[progressbar.Bar('=', '[', ']'), ' ', 
                                       progressbar.Percentage(), 
                                       " of {0}".format(num)])
bar.start()
for pred in prediction:
    final_e_nn.append(np.linalg.norm(pred))
bar.finish()

In [ ]:
energy_spectrum(final_e, 75)

In [ ]:
energy_spectrum(final_e_nn, 75)

In [ ]:
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare

mean,std=norm.fit(final_e)
plt.hist(final_e, bins=100, alpha = 0.5, label='NN prediction', density = True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
y = norm.pdf(x, mean, std)
plt.plot(x, y,'r--', linewidth=2)
plt.legend(loc='upper right')
plt.show()

In [ ]:
print('mean = ', mean)
print('std = ', std)
print("chi square = ", stats.chisquare(final_e))

In [ ]:
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare

mean_nn,std_nn=norm.fit(final_e_nn)
plt.hist(final_e_nn, bins=100, alpha = 0.5, label='NN prediction', density = True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
y = norm.pdf(x, mean_nn, std_nn)
plt.plot(x, y,'r--', linewidth=2)
plt.legend(loc='upper right')
plt.show()

In [ ]:
print('mean = ', mean_nn)
print('std = ', std_nn)
print("chi square = ", stats.chisquare(final_e_nn))

In [ ]:
plt.hist(final_e_nn, bins=100, alpha = 0.5, label='NN prediction', density = True)
plt.hist(final_e, bins=100, alpha = 0.5, label='Electron Momentum from simulations', density = True)
x_nn = np.linspace(xmin, xmax, 100)
y_nn = norm.pdf(x_nn, mean_nn, std_nn)
plt.plot(x_nn, y_nn,'r--', label = 'fit NN', linewidth = 2)
plt.legend(loc='upper right')
x_e = np.linspace(xmin, xmax, 100)
y_e = norm.pdf(x_e, mean, std)
plt.plot(x_e, y_e, 'g:', label = 'fit Electron Momentum Simulations', linewidth = 2)
plt.legend(loc = 'upper right')
plt.ylim((0, 30))
plt.savefig('Fit_250train_250test.png')
plt.show()